In [1]:
#library import
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
import os, random, json
spark=SparkSession.builder.master("local").appName("studentmarkpractice").getOrCreate()

In [2]:
class landing_to_unification:
    
    """unification"""
    
    def __init__(self,file_parameter):
        
        """function to get input parameters to read the file from local path"""
        self.file_parameter=file_parameter
        
    def read_parquet(self):
        return spark.read.format(self.file_parameter['src_format']).option("header","True").load(self.file_parameter['src_path'])
    
    def write_parquet(self,df):
        df.write.format(self.file_parameter['dest_format']).mode("overwrite").option("path",self.file_parameter['dest_path']).saveAsTable("school_unified.{0}".format(self.file_parameter['view_name']))
    
    def create_view(self,df):
        df.createOrReplaceTempView(self.file_parameter['view_name'])
    
    def open_dir(self):
        os.chdir(self.file_parameter['dest_path'])
        dest_file_path=f"{self.file_parameter['view_name']}.parquet"
        for file in os.listdir():
        # Check whether file is in text format or not
            if file.endswith(".parquet"):
                src_file_path = f"{file}"
                os.rename(src_file_path,dest_file_path)
            else:
                src_file_path = f"{file}"
                os.remove(src_file_path)
                
    def main_block(self):
        
        """main execution block"""
        df=self.read_parquet()
        self.create_view(df)
        

In [5]:
if __name__=='__main__':
    file_parameter1={
        "src_format":"parquet",
        "src_path":"C:/Git_Repo/big_data_project/data_files/landing/school_landing/student/studentdetails.parquet",
        "view_name":"studentdetails"
    }
    
    file_parameter2={
        "src_format":"parquet",
        "src_path":"C:/Git_Repo/big_data_project/data_files/landing/school_landing/subject/subjectdetails.parquet",
        "view_name":"subjectdetails"
    }
    
    file_parameter3={
        "src_format":"parquet",
        "src_path":"C:/Git_Repo/big_data_project/data_files/landing/school_landing/marks/markdetails.parquet",
        "view_name":"markdetails"
    }
    file_parameter4={
        "dest_format":"parquet",
        "dest_path":"C:/Git_Repo/big_data_project/data_files/unified/school_unified/overallreport",
        "view_name":"school"
    }
    
    spark.sql("create database if not exists {0}".format("school_unified"))
    unified_obj1=landing_to_unification(file_parameter1)
    unified_obj2=landing_to_unification(file_parameter2)
    unified_obj3=landing_to_unification(file_parameter3)
    unified_obj4=landing_to_unification(file_parameter4)
    unified_obj1.main_block()
    unified_obj2.main_block()
    unified_obj3.main_block()
    stud_df=spark.sql("SELECT markdetails.Student_ID,studentdetails.Student_Name,markdetails.Subject_ID,subjectdetails.Subject_Name,markdetails.Marks,markdetails.Date FROM((markdetails JOIN studentdetails ON markdetails.Student_ID=studentdetails.Student_ID) JOIN subjectdetails ON markdetails.Subject_ID=subjectdetails.Subject_ID)")
    unified_obj4.write_parquet(stud_df)
    unified_obj4.open_dir()

Py4JJavaError: An error occurred while calling o82.saveAsTable.
: java.io.IOException: Unable to clear output directory file:/C:/Git_Repo/big_data_project/data_files/unified/school_unified/overallreport prior to writing to it
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.deleteMatchingPartitions(InsertIntoHadoopFsRelationCommand.scala:238)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:130)
	at org.apache.spark.sql.execution.datasources.DataSource.writeAndRead(DataSource.scala:550)
	at org.apache.spark.sql.execution.command.CreateDataSourceTableAsSelectCommand.saveDataIntoTable(createDataSourceTables.scala:220)
	at org.apache.spark.sql.execution.command.CreateDataSourceTableAsSelectCommand.run(createDataSourceTables.scala:177)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.exec